In [2]:
import pandas as pd

file_path = r"C:\Users\User\OneDrive\桌面\My Course\碩一下\給台大資料\Data1-unfiltered.xlsx"  


data1_df = pd.read_excel(file_path)  
data1_df = data1_df[data1_df["車種"].str.contains("1.9T|電動車", na=False) | (data1_df["車種"] == "機車")]
data1_df["三邊加總 (CM)"] = data1_df["長"] + data1_df["寬"] + data1_df["高"]

file_path2 = r"C:\Users\User\OneDrive\桌面\My Course\碩一下\給台大資料\Data2.xlsx" 

data2_df = pd.read_excel(file_path2)
data2_df["購車金額"] = data2_df["購車金額"] / 5 / 12
data2_df.rename(columns={"購車金額": "月折舊費用 (Monthly Depreciation)"}, inplace=True)

print(data1_df.head())
print(data2_df.head())


           配送單號  作業內容                作業時間 作業人員      營業所  路線代碼 當配/一般     長  \
0  300308797073   已取貨 2024-12-14 08:39:30  人派1      桃衛1  NS16    一般  37.0   
1  300308797073   分貨中 2024-12-14 15:32:54  林宜蒨  分轉站-NS2  NS16    一般  37.0   
4  300308797073   已取貨 2024-12-15 09:00:26  張凱媛     北衛16  NS16    一般  37.0   
5  300308797073  暫存站內 2024-12-15 14:51:57  張凱媛     北衛16  NS16    一般  37.0   
6  300308797073   運送中 2024-12-15 15:50:41  徐翊桓     北衛16  NS16    一般  37.0   

      寬     高   箱號   箱型      車種   重量  三邊加總 (CM)  
0  28.0  22.0  NaN  1.0  1.9T低溫  1.0       87.0  
1  28.0  22.0  NaN  1.0  1.9T低溫  1.0       87.0  
4  28.0  22.0  NaN  1.0   電動車常溫  1.0       87.0  
5  28.0  22.0  NaN  1.0   電動車常溫  1.0       87.0  
6  28.0  22.0  NaN  1.0  1.9T常溫  1.0       87.0  
     車種  月折舊費用 (Monthly Depreciation)  油資/電費 ($/配送件數)  駕駛人員月薪  人員獎金($/配送件數) 
0  1.9T                   9500.000000            2.97   37000           12.6
1    機車                   1475.000000            0.60   31000           12.2
2  電動機

In [11]:
# Step 1: Convert 作業時間 to datetime
data1_df["作業時間"] = pd.to_datetime(data1_df["作業時間"], errors="coerce")

# Step 2: Drop rows with missing key fields
data1_clean = data1_df.dropna(subset=["作業時間", "車種", "作業人員"])

# Step 3: Normalize 1.9T車種
data1_clean["車種_normalized"] = data1_clean["車種"].replace({
    "1.9T常溫": "1.9T",
    "1.9T低溫": "1.9T"
})

# Step 4: Extract date only
data1_clean["日期"] = data1_clean["作業時間"].dt.date

# Step 5: Define unique trips (作業人員 + 車種 + 日期)
unique_trips = data1_clean.drop_duplicates(subset=["作業人員", "車種_normalized", "日期"])

# Step 6: Count trips per vehicle type (normalized)
trip_counts = unique_trips["車種_normalized"].value_counts().reset_index()
trip_counts.columns = ["車種", "trip_count"]

# Step 7: Estimate trips per month
trip_counts["estimated_trips_per_month"] = trip_counts["trip_count"] / 2.5

# Done
print(trip_counts)


      車種  trip_count  estimated_trips_per_month
0     機車        1289                      515.6
1   1.9T         620                      248.0
2  電動車常溫         316                      126.4


In [14]:
# Step 1: Normalize 1.9T types into a single category
data1_clean["車種_normalized"] = data1_clean["車種"].replace({
    "1.9T常溫": "1.9T",
    "1.9T低溫": "1.9T"
})

# Step 2: Create trip_id based on normalized 車種
data1_clean["trip_id"] = (
    data1_clean["作業人員"].astype(str) + "_" +
    data1_clean["車種_normalized"].astype(str) + "_" +
    data1_clean["日期"].astype(str)
)

# Step 3: Count unique delivery orders per trip
deliveries_per_trip = data1_clean.drop_duplicates(subset=["trip_id", "配送單號"])
deliveries_per_trip = deliveries_per_trip.groupby("trip_id").size().reset_index(name="delivery_count")

# Step 4: Map 車種 back to each trip
trip_vehicle_map = data1_clean.drop_duplicates(subset=["trip_id"])[["trip_id", "車種_normalized"]]
deliveries_per_trip = deliveries_per_trip.merge(trip_vehicle_map, on="trip_id")

# Step 5: Average number of deliveries per trip by 車種
avg_deliveries_per_trip = deliveries_per_trip.groupby("車種_normalized")["delivery_count"].mean().reset_index()
avg_deliveries_per_trip.columns = ["車種", "avg_deliveries_per_trip"]

print(avg_deliveries_per_trip)


      車種  avg_deliveries_per_trip
0   1.9T                81.961290
1     機車                58.903026
2  電動車常溫               120.797468


In [2]:
# Step 1: Drop duplicates by 配送單號 + 車種
unique_deliveries = data1_df.drop_duplicates(subset=["配送單號", "車種"])

# Step 2: Apply volume constraints on deduplicated data
mask_motorcycle = unique_deliveries["三邊加總 (CM)"] < 90
mask_e_motorcycle = unique_deliveries["三邊加總 (CM)"] < 120

# Step 3: Count total eligible deliveries
total_deliveries = len(unique_deliveries)
motorcycle_eligible = unique_deliveries[mask_motorcycle].shape[0]
e_motorcycle_eligible = unique_deliveries[mask_e_motorcycle].shape[0]
truck_only = total_deliveries - e_motorcycle_eligible

# Step 4: Print results
print(f"Total deliveries: {total_deliveries}")
print(f"機車 eligible deliveries (CM < 90): {motorcycle_eligible}")
print(f"電動機車 eligible deliveries (CM < 120): {e_motorcycle_eligible}")
print(f"1.9T only deliveries (CM ≥ 120): {truck_only}")

Total deliveries: 138807
機車 eligible deliveries (CM < 90): 102210
電動機車 eligible deliveries (CM < 120): 122918
1.9T only deliveries (CM ≥ 120): 15889


In [3]:
# Step 1: Drop duplicates by 配送單號 + 車種
unique_deliveries = data1_df.drop_duplicates(subset=["配送單號", "車種"])

# Step 2: Count how many deliveries each 車種 handled
delivery_counts = unique_deliveries["車種"].value_counts().reset_index()
delivery_counts.columns = ["車種", "total_deliveries"]

# Step 3: Estimate average deliveries per month per vehicle type
delivery_counts["estimated_deliveries_per_month"] = delivery_counts["total_deliveries"] / 2.5

# Step 1: Replace 車種 labels to unify 1.9T types
delivery_counts["車種"] = delivery_counts["車種"].replace({"1.9T低溫": "1.9T", "1.9T常溫": "1.9T"})

# Step 2: Group by the unified 車種 and sum the values
combined = delivery_counts.groupby("車種", as_index=False).sum()

# Step 3: Print the combined result
print(combined)


      車種  total_deliveries  estimated_deliveries_per_month
0   1.9T             46845                         18738.0
1     機車             58460                         23384.0
2  電動車常溫             33502                         13400.8


In [22]:
import pulp

model = pulp.LpProblem("Vehicle_Import_Optimization", pulp.LpMinimize)

# Number of vehicles to import (monthly cost base)
y_1_9t = pulp.LpVariable("y_1.9T", lowBound=0, cat="Integer")
y_emoto = pulp.LpVariable("y_電動機車", lowBound=0, cat="Integer")
y_moto = pulp.LpVariable("y_機車", lowBound=0, cat="Integer")

# Number of deliveries assigned to each vehicle type (per month)
x_1_9t = pulp.LpVariable("x_1.9T_deliveries", lowBound=0, cat="Integer")
x_emoto = pulp.LpVariable("x_電動機車_deliveries", lowBound=0, cat="Integer")
x_moto = pulp.LpVariable("x_機車_deliveries", lowBound=0, cat="Integer")

### 限制式 ###
model += (x_1_9t + x_emoto + x_moto >= 138807 / 2.5), "Total_Monthly_Delivery_Demand"
model += (x_1_9t >= 15889 / 2.5), "Monthly_Large_Item_Minimum"
model += x_1_9t <= y_1_9t * (248 * 81.96), "1.9T_Capacity"
model += x_emoto <= y_emoto * (126.4 * 120.80), "E_Moto_Capacity"
model += x_moto <= y_moto * (515.6 * 58.90), "Motorcycle_Capacity"

### 目標 ###
model += (
    y_1_9t * (9500 + 37000) + x_1_9t * (2.97 + 12.6) +
    y_emoto * (8274.92 + 31000) + x_emoto * (0.37 + 12.2) +
    y_moto * (1475 + 31000) + x_moto * (0.6 + 12.2)
), "Total_Monthly_Cost"



In [23]:
# Solve the model
model.solve()

# Show status
print("Status:", pulp.LpStatus[model.status])

# Show how many vehicles to import
print("\nOptimal number of vehicles to import:")
print("1.9T:", y_1_9t.varValue)
print("電動機車:", y_emoto.varValue)
print("機車:", y_moto.varValue)

# Show how many deliveries to assign
print("\nDeliveries assigned per month:")
print("1.9T:", x_1_9t.varValue)
print("電動機車:", x_emoto.varValue)
print("機車:", x_moto.varValue)

# Total cost
print("\nTotal Estimated Monthly Cost:", pulp.value(model.objective))


Status: Optimal

Optimal number of vehicles to import:
1.9T: 1.0
電動機車: 0.0
機車: 2.0

Deliveries assigned per month:
1.9T: 6356.0
電動機車: 0.0
機車: 49167.0

Total Estimated Monthly Cost: 839750.52
